In [20]:
import pandas as pd
# import nltk.data
from itertools import chain
import re
from collections import Counter

In [7]:
# tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')

In [14]:
data = pd.read_json('Data/boletines-AMLO.json', dtype="dict", encoding="utf-8")
boletines = data['boletines']
corpus = [boletin["contenido"] for boletin in boletines]

In [35]:
tokens = [re.findall('[a-zA-zñáéíóúüÁÉÍÓÚÜÑ]+', contenido.lower()) for contenido in corpus if len(contenido)>2]

In [36]:
counter_boletines = [Counter(contenido) for contenido in tokens]

In [45]:
ch = chain(*tokens)

In [46]:
todos_los_tokens = list(ch)

In [47]:
len(todos_los_tokens)

135638

In [49]:
df_corpus = pd.DataFrame(todos_los_tokens)

In [50]:
df_corpus.to_csv('Data/pln_corpus_amlo.csv')